In [1]:
from deepsudoku import REPO_PATH, DIFFICULTIES
import os
os.chdir(REPO_PATH)

import tensorflow as tf
import numpy as np
from deepsudoku.reinforcement_learning.ppo import *

2023-08-22 10:21:25.425924: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Sudoku Environment avaible at gymnasium as 'Sudoku-v0'.


In [2]:
envs = PPO_MultiDiscrete_Environment_Wrapper('Sudoku-v0', 5)

sudoku_input_shape = (9,9)

/home/mindset/miniconda3/envs/drl/lib/python3.11/site-packages/gymnasium/envs/registration.py:523: DeprecationWarning: WARN: The environment Sudoku-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [3]:
# Create models

# Implementation Detail 2: Orthogonal Initialization of hidden weights and constant initialization of biases and output weights
# Biases seem to be 0 by default in Keras
hidden_ortho_init = lambda: tf.keras.initializers.Orthogonal(gain=np.sqrt(2))
pol_out_ortho_init = lambda: tf.keras.initializers.Orthogonal(gain=0.01)
val_out_ortho_init = lambda: tf.keras.initializers.Orthogonal(gain=1)

def create_policy_model():
    inputs = tf.keras.Input(shape=sudoku_input_shape)
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(128, activation="tanh", kernel_initializer=hidden_ortho_init())(x)
    x = tf.keras.layers.Dense(128, activation="tanh", kernel_initializer=hidden_ortho_init())(x)
    x = [tf.keras.layers.Dense(9, activation="log_softmax", kernel_initializer=pol_out_ortho_init())(x) for i in range(3)]
    x = tf.keras.layers.Concatenate()(x)
    outputs = tf.keras.layers.Reshape((3,9))(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="pi")
    return model

def create_value_function_model():
    inputs = tf.keras.Input(shape=sudoku_input_shape)
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer=hidden_ortho_init())(x)
    #x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer=hidden_ortho_init())(x)
    outputs = tf.keras.layers.Dense(1, activation="linear", kernel_initializer=val_out_ortho_init())(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="V")
    return model


In [4]:
pi, V = PPO(envs, create_policy_model(), create_value_function_model(), TRAIN_EPOCHS=4)

2023-08-22 10:21:27.587115: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


epoch:  0  ; KL:  0  ; LR:  0.005
Collection
Tapework
epoch:  1  ; KL:  0.069094256  ; LR:  0.00375
Collection
Tapework
epoch:  2  ; KL:  0.023290897  ; LR:  0.0025
Collection
Tapework
epoch:  3  ; KL:  0.021868449  ; LR:  0.00125
Collection
Tapework
